In [128]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import librosa
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
DATADIR = "data/"
CATEGORIES = ["Segmented_Laugh", "Segmented_NonLaugh4"]

# Loading Data

In [269]:
training_data = []
def create_training_data():
    count=0
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES.index(category)
        for aud in os.listdir(path):
            if aud == '.DS_Store':
                continue
            aud_array , sr = librosa.load(os.path.join(path,aud), sr=None)
            count+=1
            mfccs = librosa.feature.mfcc(aud_array, sr=sr)
            training_data.append([mfccs[0:6,:].reshape(-1,1), class_num])
    print(count)
            
create_training_data()
data = np.array(training_data)

4639


# Shuffling data

In [270]:
# mfcc array | class
random.shuffle(training_data)

In [271]:
# each row has 96 features(mfcc matrix)
data[0,0].shape

(96, 1)

In [272]:
X = []
Y = []
for features, label in training_data:
    X.append(features)
    Y.append(label)

    
X = np.array(X).reshape(-1,16,6)
#-1 corresponds to how many features we have
Y = np.array(Y)
#X = X/250
#print(X.shape)
#print(Y)
X.shape

(4639, 16, 6)

# Merging 4639 rows 

In [273]:
q=[]
for s in X:
    for c in s:
        q.append(c)

In [274]:
np.array(q).shape

(74224, 6)

# Updating y for each 16 x 

In [275]:
m=[]
for s in Y:
    for c in range(16):
        m.append(s)
        

In [276]:
np.array(m).shape

(74224,)

# Data split for training

In [277]:
X_train, X_test, y_train, y_test = train_test_split(q, m, 
                                                    test_size=0.1, 
                                                    random_state=4)

# Training SVM 

In [278]:
clf = svm.SVC(gamma='auto',C=50)
clf.fit(X_train,y_train)

SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Testing it against test dataset 

In [279]:
Y_pre=[]
for mfcc in X_test:
    Y_pre.append(clf.predict(np.array([mfcc])))

In [280]:
accuracy_score(y_test,Y_pre)

0.5813013606358615